In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame

In [2]:
STable1: DataFrame = pd.read_csv("STable1_all_genomes_info.tsv", sep="|")
STable2: DataFrame = pd.read_csv("STable2_BiG-SLICE_t0.4_GCF_assignment.csv", sep=",")
STable3: DataFrame = pd.read_csv("STable3_BiG-SLICE_t0.5_GCF_assignment.csv", sep=",")
STable4: DataFrame = pd.read_csv("STable4_BiG-SLICE_t0.6_GCF_assignment.csv", sep=",")
STable5: DataFrame = pd.read_csv("STable5_BiG-SLICE_t0.7_GCF_assignment.csv", sep=",")

In [3]:
STable1 = STable1.rename(columns={"bgc_ids":"bgc_id"})

STable1["bgc_id"] =  STable1.bgc_id.str.split(",")

STable1 = STable1.explode(column="bgc_id").reset_index(drop=True)

STable1["bgc_id"] = STable1.bgc_id.astype("int64")

thresholds_4_gcfs = pd.merge(STable1, STable2, on='bgc_id')

taxonomy = thresholds_4_gcfs.taxonomy.str.split(",",expand=True).fillna("")

streptomyces_index = taxonomy[taxonomy[5].str.startswith("Streptomyces")].index

thresholds_4_gcfs.loc[streptomyces_index].gcf_id.value_counts()

gcf_id
57228    1074
49265     786
45969     687
42802     672
883       592
         ... 
63864       1
33437       1
37103       1
26535       1
4696        1
Name: count, Length: 8930, dtype: int64

In [4]:
thresholds_5_gcfs = pd.merge(STable1, STable3, on='bgc_id')

taxonomy_5 = thresholds_5_gcfs.taxonomy.str.split(",",expand=True).fillna("")
streptomyces_index_5 = taxonomy_5[taxonomy_5[5].str.startswith("Streptomyces")].index

thresholds_5_gcfs.loc[streptomyces_index_5].gcf_id.value_counts()

gcf_id
40575    2243
32610    1383
2288      661
584       596
12007     588
         ... 
47241       1
29004       1
34112       1
5342        1
42805       1
Name: count, Length: 6980, dtype: int64

In [5]:
thresholds_6_gcfs = pd.merge(STable1, STable4, on='bgc_id')

taxonomy_6 = thresholds_6_gcfs.taxonomy.str.split(",",expand=True).fillna("")
streptomyces_index_6 = taxonomy_6[taxonomy_6[5].str.startswith("Streptomyces")].index

thresholds_6_gcfs.loc[streptomyces_index_6].gcf_id.value_counts()

gcf_id
31953    1989
22441    1939
2101     1482
18765    1482
1488     1362
         ... 
9010        1
15161       1
14992       1
31676       1
31238       1
Name: count, Length: 5280, dtype: int64

In [6]:
thresholds_7_gcfs = pd.merge(STable1, STable5, on='bgc_id')

taxonomy_7 = thresholds_7_gcfs.taxonomy.str.split(",",expand=True).fillna("")
streptomyces_index_7 = taxonomy_7[taxonomy_7[5].str.startswith("Streptomyces")].index

thresholds_7_gcfs.loc[streptomyces_index_7].gcf_id.value_counts()

gcf_id
9710     3978
3703     2422
15483    1630
1951     1495
12708    1482
         ... 
22647       1
2083        1
17759       1
17890       1
19295       1
Name: count, Length: 3464, dtype: int64

# END OF ADDITIONAL EFFORTS

In [7]:
# thresholds_4_gcfs: np.ndarray = np.array(STable2.gcf_id.unique())
# thresholds_5_gcfs: np.ndarray = np.array(STable3.gcf_id.unique())
# thresholds_6_gcfs: np.ndarray = np.array(STable4.gcf_id.unique())
# thresholds_7_gcfs: np.ndarray = np.array(STable5.gcf_id.unique())
# bgc_ids: DataFrame = STable1.bgc_ids.str.split(",", expand=True)
# bgc_ids = (bgc_ids.apply(pd.to_numeric, downcast="unsigned"))
# bgc_ids[["dataset_name", "AccNo", "taxonomy"]] = STable1[["dataset_name", "AccNo", "taxonomy"]]
# STable2["bgc_id"] = STable2["bgc_id"].astype(np.int64)
# STable2.index = STable2["bgc_id"]
# STable2["taxonomy"] = ""
# from pandarallel import pandarallel
# import os
# pandarallel.initialize(nb_workers=os.cpu_count())
# def add_taxonomy_to_table(gcfs_array: np.ndarray, stable: DataFrame) -> None:
#     # print(type(gcfs_array))
#     # print(gcfs_array[5])
#     for gcf in gcfs_array:
#         # print(f"gcf{gcf}\n")
#         bgc_ids_of_gcf: np.ndarray = np.array(STable2[STable2.gcf_id == gcf]["bgc_id"])
#         for bgc_id in bgc_ids_of_gcf:
#             # print(f"bgc_id{bgc_id}\n")
#             # print(f"type(bgc_id{type(bgc_id)}\n")
#             bgc_tax = bgc_ids[bgc_ids.eq(bgc_id).any(axis="columns")]["taxonomy"].iloc[0]
#             stable.loc[bgc_id, "taxonomy"] = bgc_tax
#add_taxonomy_to_table = np.vectorize(add_taxonomy_to_table)

#add_taxonomy_to_table(gcfs_array = thresholds_4_gcfs[-10:], stable = STable2)